In [1]:
from generate_decision_tree_visualization_data import *

In [2]:
# Import necessary libraries
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Load the Iris dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)

# Train the classifier
clf = clf.fit(X_train, y_train)


In [3]:
save_tree_to_json(extract_tree_structure(clf, iris.feature_names, iris.target_names), filename="test.json")

the code works for the iris dataset generated tree, let's try a bigger one

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv ("test_dataset.csv")

In [6]:
df.columns

Index(['y_', 'ID', 'engine_age', 'length', 'power', 'month', 'landing',
       'weight', 'value', 'value_cpi', 'price', 'y_month', 'year', 'patch',
       'dist', 'patch_area', 'weight_lym', 'weight_lm', 'val_lm', 'val_lym',
       'nao_index', 'surf_temp'],
      dtype='object')

In [7]:
for col in list (df.columns):
    print (col, ":", df[col].values)

y_ : [1. 1. 0. ... 1. 0. 1.]
ID : [1993001257 1993005128 1996007882 ... 2007039109 2001015121 2008043528]
engine_age : [10. 26. 32. ...  3.  9.  8.]
length : [10.5  21.3  12.13 ...  7.   14.16 35.49]
power : [ 367.  970.  190. ...  144.  291. 1000.]
month : [ 44 131 170 ... 124 121 198]
landing : ['RISØR' 'BÅTSFJORD' 'ØKSNES' ... 'BØMLO' 'RØST' 'MÅSØY']
weight : [5.0000e+00 1.9000e+01 0.0000e+00 ... 1.5000e+00 0.0000e+00 9.4625e+04]
value : [9.5700000e+01 5.6459000e+02 0.0000000e+00 ... 1.9170000e+01 0.0000000e+00
 5.0858288e+05]
value_cpi : [1.1962000e+02 6.1169000e+02 0.0000000e+00 ... 2.0680000e+01 0.0000000e+00
 4.8996424e+05]
price : [19.14       29.71526316  0.         ... 12.78        0.
  5.37472   ]
y_month : [ 8 11  2 ...  4  1  6]
year : [ 3. 10. 14. ... 10. 10. 16.]
patch : ['09-16' '03-03' '04-27' ... '08-15' '07-26' '03-07']
dist : [ 13.6  59.6 286.7 ...  10.6 457.5 220.2]
patch_area : [3229 2122 4596 ... 1418 5377 2768]
weight_lym : [0. 0. 0. ... 1. 0. 0.]
weight_lm : [1

In [8]:
#remove 
# non numerical data 
# value adjusted for inflation (check dataset page for more info)
# other non relevant/not known features 
df.drop(["landing", "patch", "value_cpi", "y_", "ID", "dist", "patch_area", "weight_lym", "weight_lm", "val_lm", "val_lym", "nao_index", "price"], axis=1, inplace=True)

In [9]:
df.columns

Index(['engine_age', 'length', 'power', 'month', 'weight', 'value', 'y_month',
       'year', 'surf_temp'],
      dtype='object')

In [10]:
y = df["value"]
X = df[['engine_age', 'length', 'power', 'month', 'weight', 'y_month',
       'year', 'surf_temp']]

In [11]:
#make y categorical
labels = [
    "Poor Session", "Below Average", "Average Session", "Above Average", 
    "Good Session", "Great Session", "Excellent Session", 
    "Outstanding", "Legendary", "Epic"
]
# Split the values into 10 categories with meaningful labels
y = pd.cut(y, bins=10, labels=labels)

In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a Decision Tree Classifier
clf2 = DecisionTreeClassifier(random_state=42)

# Train the classifier
clf2 = clf.fit(X_train, y_train)

In [13]:
save_tree_to_json(extract_tree_structure(clf, feature_names=['engine_age', 'length', 'power', 'month', 'weight', 'y_month',
       'year', 'surf_temp'], target_names=labels), filename="fishingTree.json")